In [243]:
# Define Imports
import os
import googlemaps
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import pydeck as pdk
from dash import Dash, dcc, html
from dotenv import load_dotenv
import numpy as np
import requests
import dash_deck
import dash
from dash import Output, Input

In [244]:
# Load ENV variables
load_dotenv(dotenv_path='../.env')

True

In [245]:
# Read Golden Submissions Dataset
submissions_golden_df = pd.read_csv("../data/submissions_golden.csv")
submissions_golden_df

,initiator_region,item_classification,submission_status,number_of_submissions,latitude,longitude,latitude_grid_item_classification,longitude_grid_item_classification,color,latitude_grid_submission_status,longitude_grid_submission_status
0,Lviv,kitchen,pending,21,49.839683,24.029717,49.999683,23.869717,NaN,NaN,NaN
1,Lviv,kitchen,withdrawn,5,49.839683,24.029717,49.999683,23.869717,NaN,NaN,NaN
2,Lviv,kitchen,approval,4,49.839683,24.029717,49.999683,23.869717,NaN,NaN,NaN
3,Lviv,kitchen,declined,1,49.839683,24.029717,49.999683,23.869717,NaN,NaN,NaN
4,Lviv,shelter,pending,2,49.839683,24.029717,49.999683,24.029717,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
239,Transcarpathian,kitchen,approval,0,48.620800,22.287883,NaN,NaN,"[0, 0, 0]",48.7808,21.967883
240,Transcarpathian,shelter,withdrawn,1,48.620800,22.287883,NaN,NaN,NaN,48.7808,22.127883
241,Transcarpathian,kitchen,active,0,48.620800,22.287883,NaN,NaN,"[0, 0, 0]",48.7808,22.287883
242,Transcarpathian,kitchen,unsuccessful,0,48.620800,22.287883,NaN,NaN,"[0, 0, 0]",48.7808,22.447883


In [246]:
# Read Raw Submissions Dataset (Needed for a time-line plot)
submissions_raw_df = pd.read_csv("../data/submissions.csv")
submissions_raw_df = submissions_raw_df.dropna()
submissions_raw_df.loc[:, 'submission_date'] = pd.to_datetime(submissions_raw_df.loc[:, 'submission_date'])
submissions_raw_df

,submission_code,submission_date,submission_status,submission_description,initiator_id,initiator_legalname,initiator_address,initiator_region,item_amount,item_classification
0,010624-040A8668,2024-06-01 17:36:27.658298+00:00,pending,Харчоблок базової кухні,4394869,ВЕЛИКОБІРКІВСЬКА СЕЛИЩНА РАДА,"Тернопільська обл., Тернопільський р-н, селище...",Тернопільська,3967203.0,kitchen
1,020624-2C34FAB7,2024-06-02 16:32:18.130273+00:00,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,9607874.0,kitchen
2,020624-9F163A6A,2024-06-02 18:06:32.388695+00:00,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,16569763.0,kitchen
3,020624-D8610EBB,2024-06-02 17:09:51.508281+00:00,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,20051019.0,kitchen
4,020624-E1B2F3B0,2024-06-02 16:20:55.533024+00:00,pending,харчоблок,2146222,"УПРАВЛІННЯ ОСВІТИ, МОЛОДІ ТА СПОРТУ ЛОЗІВСЬКОЇ...","Харківська обл., місто Лозова, МІКРОРАЙОН 1, б...",Харківська,16166182.0,kitchen
...,...,...,...,...,...,...,...,...,...,...
483,310524-F8C525A4,2024-05-31 11:04:36.688246+00:00,withdrawn,Харчоблок Сварицевицького ліцею,44153430,"ВІДДІЛ ОСВІТИ, КУЛЬТУРИ, ТУРИЗМУ, МОЛОДІ ТА СП...","Рівненська обл., Дубровицький р-н, місто Дубро...",Рівненська,8327800.0,kitchen
484,310524-F9A91731,2024-05-31 15:20:47.521727+00:00,approval,Капітальний ремонт харчоблоку (їдальні) ліцею ...,44084980,ВІДДІЛ ОСВІТИ ПІДГОРОДНЕНСЬКОЇ МІСЬКОЇ РАДИ ДН...,"Дніпропетровська обл., Дніпровський р-н, місто...",Дніпропетровська,11745067.5,kitchen
485,310524-F9C0426D,2024-05-31 18:11:46.185941+00:00,pending,Капітальний ремонт харчоблоку Середньоберезівс...,4354255,ЯБЛУНІВСЬКА СЕЛИЩНА РАДА КОСІВСЬКОГО РАЙОНУ ІВ...,"Івано-Франківська обл., Косівський р-н, селище...",Івано-Франківська,870410.0,kitchen
486,310524-FD0808D6,2024-05-31 11:34:02.120912+00:00,withdrawn,Капітальний ремонт харчоблоку,4412567,ПОНОРНИЦЬКА СЕЛИЩНА РАДА,"Чернігівська обл., Новгород-Сіверський р-н, се...",Чернігівська,3122988.0,kitchen


In [247]:
# Define a tooltip for map
tooltip = {
    'html': '<b>Initiator Region:</b> {initiator_region}<br>'
            '<b>{tooltip_title}:</b> {tooltip_category}<br>'
            '<b>Number of Submissions:</b> {number_of_submissions}',
    'style': {
        'color': 'white',
        'z-index': 99
    }
}

In [248]:
# Define plot parameters
BARPLOT_HEIGHT: int = 400
LINEPLOT_HEIGHT: int = 300
PYDECK_STATUS_MAP_COLOR_MAP: dict = {
    'approval': [72, 143, 49],
    'active': [142, 174, 84],
    'pending': [202, 205, 128],
    'withdrawn': [247, 188, 125],
    'unsuccessful': [239, 131, 95],
    'declined': [222, 66, 91],
}
# Transform rgb list which PyDeck use to RGB format for plotly
# Henceforth, we stay consistent and easy to update
PLOTLY_STATUS_BARPLOT_COLOR_MAP: dict = {
    k: f"rgb({r}, {g}, {b})" for k, (r, g, b) in PYDECK_STATUS_MAP_COLOR_MAP.items()
}
PYDECK_ITEM_CLASSIFICATION_COLOR_MAP: dict = {
    'kitchen': [124, 198, 116],
    'shelter': [115, 197, 197]
}
PLOTLY_ITEM_CLASSIFICATION_COLOR_MAP: dict = {
    k: f"rgb({r}, {g}, {b})" for k, (r, g, b) in PYDECK_ITEM_CLASSIFICATION_COLOR_MAP.items()
}

In [249]:
# Add GeoJSON layer to visualize admin units
geojson_url = "https://github.com/wmgeolab/geoBoundaries/raw/905b0ba/releaseData/gbOpen/UKR/ADM1/geoBoundaries-UKR-ADM1_simplified.geojson"
response = requests.get(geojson_url)
ukraine_geojson = response.json()

geojson_layer = pdk.Layer(
    'GeoJsonLayer',
    data=ukraine_geojson,
    stroked=True,
    filled=True,
    get_line_color=[255, 255, 255],
    get_line_width=1000,
    get_fill_color=[0, 0, 0, 20],
    wireframe=True,
    extruded=False,
)

In [250]:
# Prepare a placeholders for a map
# Set the viewport location
view_state = pdk.ViewState(
    latitude=submissions_golden_df['latitude'].mean(),
    longitude=submissions_golden_df['longitude'].mean() + 4,
    zoom=5,
    pitch=33
)
# Create the initial Deck object
initial_deck = pdk.Deck(
    layers=[geojson_layer],
    initial_view_state=view_state,
    tooltip=tooltip,
    map_style='mapbox://styles/mapbox/dark-v10',
)

# Create the DeckGL component
deck_component = dash_deck.DeckGL(
    initial_deck.to_json(),
    id='deck-gl',
    tooltip=tooltip,
    mapboxKey=os.getenv("MAPBOX_API_KEY"),
    style={"height": '42.5vh', "position": 'relative'}
)

In [251]:
# Prepare Bar Plot and Annotations
# Create the bar plot
status_counts = submissions_golden_df.loc[:,
                    'submission_status'
                ].value_counts(
                ).reset_index()
status_counts.columns = ['submission_status', 'count']
status_fig = px.bar(
    status_counts,
    x='submission_status',
    y='count',
    title='Distribution of Submissions by Status',
    labels={
        'submission_status': 'Submission Status',
        'count': 'Count'
    },
    color='submission_status',
    color_discrete_map=PLOTLY_STATUS_BARPLOT_COLOR_MAP,
    height=BARPLOT_HEIGHT,
    template='plotly_dark',
)
status_fig.update_layout(
    legend_title_text='Submission Status',
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    margin=dict(r=150),
    font=dict(
        family="Helvetica",
        size=14,
        color="white"
    )
)
status_fig.show()

In [252]:
# Prepare Bar Plot for Item Classification
item_classification_counts = submissions_golden_df['item_classification'].value_counts().reset_index()
item_classification_counts.columns = ['item_classification', 'count']

# Create the bar plot
item_classification_fig = px.bar(
    item_classification_counts,
    x='item_classification',
    y='count',
    title='Distribution of Submissions by Item Classification',
    labels={
        'item_classification': 'Item Classification',
        'count': 'Count'
    },
    color='item_classification',
    color_discrete_map={
        'kitchen': 'rgb(124, 198, 116)',
        'shelter': 'rgb(115, 197, 197)'
    },
    height=BARPLOT_HEIGHT,
    template='plotly_dark',
)

item_classification_fig.update_layout(
    legend_title_text='Item Classification',
    legend=dict(
        x=1.05,
        y=1,
        traceorder='normal',
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    margin=dict(r=150),
    font=dict(
        family="Helvetica",
        size=14,
        color="white"
    )
)

item_classification_fig.show()

In [253]:
# Create time-series figure
time_series_fig = px.line(
    submissions_raw_df.groupby(pd.Grouper(key='submission_date', freq='D')).size().reset_index(name='count'),
    x='submission_date',
    y='count',
    title='Number of Daily Submissions',
    height=LINEPLOT_HEIGHT,
    markers=True,
    labels={
        'submission_date': 'Submission Date',
        'count': 'Count'
    },
)
time_series_fig.data[0].line.color = 'skyblue'
time_series_fig.show()

/Users/rkovalch/Library/Caches/pypoetry/virtualenvs/data-viz-final-project-wMDNdqrB-py3.12/lib/python3.12/site-packages/pandas/core/groupby/grouper.py:381: FutureWarning:

Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.



In [254]:
# Create Annotations section
annotations_section_status = html.Div([
    html.H3("Annotations", style={'font-weight': 'bold'}),
    html.P("This plot shows the distribution of submissions by status for financing educational facilities. "
           "The data is provided by the DREAM API. Below are the categories of submission status with their respective colors:"),
    html.Ul([
        html.Li([html.Span("■ ", style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP["approval"]}), "Approval - Submission has been approved."]),
        html.Li([html.Span("■ ", style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP["active"]}), "Active - Submission is currently active."]),
        html.Li([html.Span("■ ", style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP["pending"]}), "Pending - Submission is awaiting review."]),
        html.Li([html.Span("■ ", style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP["withdrawn"]}), "Withdrawn - Submission was withdrawn by the applicant."]),
        html.Li([html.Span("■ ", style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP["unsuccessful"]}), "Unsuccessful - Submission was not successful."]),
        html.Li([html.Span("■ ", style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP["declined"]}), "Declined - Submission was declined."])
    ], style={'list-style-type': 'none', 'padding-left': '20px'}),
    html.Div([
    html.P("Moreover, we split the submissions by two groups as a gradation: from more successful, and green, like active (", style={'display': 'inline'}),
    html.Span('■', style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP['approval'], 'display': 'inline'}),
    html.P(") to more red, as for example declined: (", style={'display': 'inline'}),
    html.Span('■', style={'color': PLOTLY_STATUS_BARPLOT_COLOR_MAP['declined'], 'display': 'inline'}),
    html.P(")", style={'display': 'inline'})
]),
], style={'color': 'white', 'font-family': 'Arial', 'font-size': '14px', 'padding': '20px', 'background-color': 'black', 'position': 'relative'})
annotations_section_classification = html.Div([
    html.H3("Annotations", style={'font-weight': 'bold'}),
    html.P("This plot shows the distribution of submissions by type for financing educational facilities. "
           "The data is provided by the DREAM API. Below are the categories of submission type with their respective colors:"),
    html.Ul([
        html.Li([html.Span("■ ", style={'color': PLOTLY_ITEM_CLASSIFICATION_COLOR_MAP["kitchen"]}), "Kitchen - Submission for kitchen facilities."]),
        html.Li([html.Span("■ ", style={'color': PLOTLY_ITEM_CLASSIFICATION_COLOR_MAP["shelter"]}), "Shelter - Submission for shelter facilities."])
    ], style={'list-style-type': 'none', 'padding-left': '20px'}),
], style={'color': 'white', 'font-family': 'Arial', 'font-size': '14px', 'padding': '20px', 'background-color': 'black', 'position': 'relative'})


In [255]:
def adjust_color(color, alpha=50):
    """Adjust alpha of given color."""

    return color[:3] + [alpha]

# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define App Layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H1("DREAM Dashboard"),
                html.H3("Submissions Grouped by Region"),
                html.Hr(),
            ], style={'background-color': 'black', 'color': 'white', 'padding-left': '20px', 'padding-top': '10px'}),
        ], md=12),
        dbc.Col([
            html.Div([
                html.H4("Select Group By Option:", style={"color": "white"}),
                dcc.Dropdown(
                    id='groupby-dropdown',
                    options=[
                        {'label': 'Item Classification', 'value': 'item_classification'},
                        {'label': 'Submission Status', 'value': 'submission_status'},
                    ],
                    value='item_classification',
                    clearable=False,
                ),
            ], style={'background-color': 'black', 'color': 'black', 'padding-left': '20px'}),
            # Annotations
            html.Div(id='annotations-container'),
        ], md=5),
        dbc.Col([
            # BarPlot
            dcc.Graph(id='barplot'),
        ], md=7),
        dbc.Col([
            html.Div(
                deck_component,
                #style={'width': '100%', 'height': '10vh'}
            )
        ], md=12),
        dbc.Col([
            # Time-Series Selector
            dcc.Graph(figure=time_series_fig),
        ], md=12),
    ])
], fluid=True, style={"background-color": "black"})

@app.callback(
    [Output('annotations-container', 'children'),
     Output('barplot', 'figure')],
    [Input('groupby-dropdown', 'value')]
)
def update_layout(group_by):
    if group_by == 'item_classification':
        annotations = annotations_section_classification

        # Prepare Bar Plot for Item Classification
        item_classification_counts = submissions_golden_df['item_classification'].value_counts().reset_index()
        item_classification_counts.columns = ['item_classification', 'count']

        # Create the bar plot
        fig = px.bar(
            item_classification_counts,
            x='item_classification',
            y='count',
            title='Distribution of Submissions by Item Classification',
            labels={
                'item_classification': 'Item Classification',
                'count': 'Count'
            },
            color='item_classification',
            color_discrete_map={
                'kitchen': 'rgb(124, 198, 116)',
                'shelter': 'rgb(115, 197, 197)'
            },
            height=BARPLOT_HEIGHT,
            template='plotly_dark',
        )
        fig.update_layout(
            legend_title_text='Item Classification',
            legend=dict(
                x=1.05,
                y=1,
                traceorder='normal',
                bgcolor='rgba(255, 255, 255, 0)',
                bordercolor='rgba(255, 255, 255, 0)'
            ),
            margin=dict(r=150),
            font=dict(
                family="Helvetica",
                size=14,
                color="white"
            )
        )

    elif group_by == 'submission_status':
        annotations = annotations_section_status

        # Prepare Bar Plot for Submission Status
        status_counts = submissions_golden_df['submission_status'].value_counts().reset_index()
        status_counts.columns = ['submission_status', 'count']

        # Create the bar plot
        fig = px.bar(
            status_counts,
            x='submission_status',
            y='count',
            title='Distribution of Submissions by Status',
            labels={
                'submission_status': 'Submission Status',
                'count': 'Count'
            },
            color='submission_status',
            color_discrete_map=PLOTLY_STATUS_BARPLOT_COLOR_MAP,
            height=BARPLOT_HEIGHT,
            template='plotly_dark',
        )
        fig.update_layout(
            legend_title_text='Submission Status',
            legend=dict(
                x=1.05,
                y=1,
                traceorder='normal',
                bgcolor='rgba(255, 255, 255, 0)',
                bordercolor='rgba(255, 255, 255, 0)'
            ),
            margin=dict(r=150),
            font=dict(
                family="Helvetica",
                size=14,
                color="white"
            )
        )

    return annotations, fig

@app.callback(
Output('deck-gl', 'data'),
Input('groupby-dropdown', 'value')
)
def update_map(group_by):
    if group_by == 'item_classification':
        lat_col = f'latitude_grid_{group_by}'
        long_col = f'longitude_grid_{group_by}'
        grouped_df = submissions_golden_df.groupby(
            ['initiator_region', group_by],
            as_index=False
        ).agg({
            'number_of_submissions': 'count',
            lat_col: 'first',
            long_col: 'first',
            'color': 'first'
        })
        color_map = PYDECK_ITEM_CLASSIFICATION_COLOR_MAP
        grouped_df['color'] = grouped_df.apply(
            lambda row: adjust_color(
                color_map[row['item_classification']],
                alpha=75
            ) if row['number_of_submissions'] == 0 else color_map[row['item_classification']],
            axis=1
        )

    elif group_by == 'submission_status':
        lat_col = f'latitude_grid_{group_by}'
        long_col = f'longitude_grid_{group_by}'
        grouped_df = submissions_golden_df.groupby(
        ['initiator_region', group_by],
        as_index=False
        ).agg({
        'number_of_submissions': 'count',
        lat_col: 'first',
        long_col: 'first',
        'color': 'first'
        })
        color_map = PYDECK_STATUS_MAP_COLOR_MAP
        grouped_df['color'] = grouped_df.apply(
            lambda row: adjust_color(
                color_map[row['submission_status']],
                alpha=75
            ) if row['number_of_submissions'] == 0 else color_map[row['submission_status']],
            axis=1
        )

    grouped_df = grouped_df.loc[:, [
        "initiator_region",
        "number_of_submissions",
        "color",
        lat_col,
        long_col
    ] + [group_by]].dropna()
    grouped_df.loc[:, "tooltip_title"] = group_by.replace("_", " ").title()
    grouped_df.loc[:, "tooltip_category"] = grouped_df.loc[:, group_by].str.title()

    # Create the layer
    layer = pdk.Layer(
        'ColumnLayer',
        elevation_scale=5000,
        data=grouped_df,
        get_position=[long_col, lat_col],
        get_fill_color='color',
        get_elevation="number_of_submissions",
        radius=800,
        radius_scale=3000,
        radius_min_pixels=1,
        radius_max_pixels=500,
        pickable=True,
        auto_highlight=True,
        extruded=True,
        coverage=7,
        elevation_range=[0, 5000],
        tooltip=tooltip,
    )

    deck = pdk.Deck(
        layers=[layer, geojson_layer],
        initial_view_state=view_state,
        tooltip=tooltip,
        map_style='mapbox://styles/mapbox/dark-v10',
    )

    return deck.to_json()

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)

In [256]:
submissions_golden_df.loc[submissions_golden_df.loc[:, "initiator_region"] == "Херсонська"]

,initiator_region,item_classification,submission_status,number_of_submissions,latitude,longitude,latitude_grid_item_classification,longitude_grid_item_classification,color,latitude_grid_submission_status,longitude_grid_submission_status


In [257]:
group_by = "submission_status"
lat_col = "latitude_grid_item_classification"
long_col = "longitude_grid_item_classification"

submissions_golden_df.groupby(
            ['initiator_region', group_by],
            as_index=False
            ).agg({
            'number_of_submissions': 'count',
            lat_col: 'first',
            long_col: 'first',
        })

,initiator_region,submission_status,number_of_submissions,latitude_grid_item_classification,longitude_grid_item_classification
0,Cherkasy,active,1,NaN,NaN
1,Cherkasy,approval,1,NaN,NaN
2,Cherkasy,declined,1,NaN,NaN
3,Cherkasy,pending,4,49.604433,31.899767
4,Cherkasy,unsuccessful,1,NaN,NaN
...,...,...,...,...,...
127,Zhytomyr,approval,2,50.414650,28.498667
128,Zhytomyr,declined,1,NaN,NaN
129,Zhytomyr,pending,3,50.414650,28.498667
130,Zhytomyr,unsuccessful,1,NaN,NaN


In [258]:
submissions_golden_df.loc[submissions_golden_df.loc[:, 'initiator_region'] == 'Рівненська']

,initiator_region,item_classification,submission_status,number_of_submissions,latitude,longitude,latitude_grid_item_classification,longitude_grid_item_classification,color,latitude_grid_submission_status,longitude_grid_submission_status
